# TP1 - Programmation Python 

In [1]:
import sys
print(sys.argv[0])

/home/alexis/.local/lib/python3.10/site-packages/ipykernel_launcher.py


> NB: N'hésitez pas à réutiliser des fonctions d'une partie à l'autre !

## Analyse de texte : listes, dictionnaires, ensembles

Afin de gagner en familiarité avec les structures de données Python les plus utiles, vous allez
développer des fonctions pour analyser le vocabulaire utilisé dans un texte. 
Vous trouverez sur ma page une version d'Alice aux pays des merveilles, mais vous pouvez récupérer le texte
de votre choix. J'ai choisi un texte an anglais pour simplifier les problèmes d'accents. 

http://www.irit.fr/~Philippe.Muller/alice_wonderland.utf8.txt

### Première approche simple

Définissez un ensemble de fonctions pour lire un texte d'un fichier, et compter le nombre d'occurrence de chaque mot.  
Il faudra bien sûr gérer la ponctuation, entre autres.  
Le but est de sortir les mots les plus significatifs utilisés dans l'oeuvre considérée.  
Quels sont les mots les plus fréquents dans le texte ? Qu'en pensez vous ? 

In [15]:
class TextTokens:

    # Attributes
    ponct = set(",;?.:-!&'()")
    stop_words = set("one the a some no not is be are has and to it she i of said you in that as her t at s on all with had but for they so vey what he there if his then this them were would was herself do have when or could went off me into see how well m did went about know like can your who don now * my by began ll its thought an their just say".split())  # Split stop keywords to remove from the text

    def __init__(self, filename):
        self.file = open(filename).read().strip()

    @staticmethod
    def comptage(liste_items):
        res = {}
        for i in liste_items:
            res[i] = res.get(i, 0) + 1
        return res

    @staticmethod
    def normalise(mot):
        res = mot.strip().replace("'", "").lower()
        return res

    def enleve_ponctuation(self, text):
        res = text
        for sign in self.ponct:
            res = res.replace(sign, " ")
        return res

    def list_tokens(self, text):
        new_text = self.enleve_ponctuation(text)
        mots = [self.normalise(mot) for mot in new_text.split() if self.normalise(mot) not in self.stop_words]
        return mots

    def stats_text(self, text):
        mots = self.list_tokens(text)
        return self.comptage(mots)

    def sorted_stat(self):
        stats = self.stats_text(self.file)
        sorted_stats = [(y, x) for (x, y) in list(stats.items())]
        sorted_stats.sort()
        sorted_stats.reverse()
        return sorted_stats

    def display(self, sorted_stats):
        print("Tokens:")
        print(sorted_stats[:50])

doc = TextTokens("alice_wonderland_utf8.txt")
doc.display(doc.sorted_stat())


Tokens:
[(398, 'alice'), (144, 'very'), (128, 'little'), (117, 'out'), (102, 'down'), (99, 'up'), (83, 'again'), (75, 'queen'), (71, 'time'), (63, 'king'), (58, 'turtle'), (56, 'way'), (56, 'mock'), (56, 'hatter'), (55, 'quite'), (55, 'gryphon'), (53, 'think'), (51, 'rabbit'), (51, 'here'), (51, 'first'), (50, 'only'), (50, 'much'), (50, 'head'), (50, 'go'), (49, 'which'), (49, 'thing'), (49, 'more'), (48, 'voice'), (47, 'never'), (46, '"'), (45, 'looked'), (45, 'got'), (45, 'get'), (44, 've'), (44, 'oh'), (44, 'must'), (44, 'mouse'), (44, 'come'), (43, 'him'), (43, 'after'), (42, 'duchess'), (41, 'round'), (40, 'why'), (40, 'two'), (40, 'tone'), (40, 'such'), (40, 'over'), (40, 'other'), (40, 'dormouse'), (40, 'came')]


### Texte prétraité

Vous avez du remarquer entre autres problèmes, que certains mots que l'on voudrait regrouper apparaissent sous des formes différentes (pluriel des noms, verbes conjugués), et que les mots fonctionnels (déterminants, prépositions par exemple) sont courants sans être très intéressants.  
Vous trouverez dans le fichier [alice_wonderland.utf8.conll](http://www.irit.fr/~Philippe.Muller/alice_wonderland.utf8.conll) une version du texte déjà prétraité, où chaque ligne correspond à une analyse prélable d'un mot du texte, avec sa forme telle qu'elle apparait dans le texte, son lemme (cad la forme normalisée correspondant à son entrée dans le dictionnaire), et une étiquette donnant sa catégorie: nom, verbe, déterminant, etc.  

**Ecrivez de nouvelles fonctions pour refaire les analyses précédentes de façon plus simple avec ce fichier, en essayant de paramétrer le plus possible (faire varier les catégories à garder par exemple).**

In [36]:
class PreprocessedTextTokens:

    def __init__(self, filename):
        self.file = open(filename, 'r', encoding='utf-8').readlines()

    def list_tokens(self, categories_to_keep=None):
        tokens = []
        for line in self.file:
            parts = line.strip().split('\t')
            if len(parts) == 3:
                word, category, lemma = parts
                if categories_to_keep is None or category in categories_to_keep:
                    tokens.append((word, lemma))
        return tokens

    def count_tokens(self, tokens):
        token_count = {}
        for token, lemma in tokens:
            token_count[token] = token_count.get(token, 0) + 1
        return token_count

    def sorted_token_stat(self, categories_to_keep=None):
        tokens = self.list_tokens(categories_to_keep)
        token_count = self.count_tokens(tokens)
        sorted_stats = [(count, token) for token, count in token_count.items()]
        sorted_stats.sort(reverse=True)
        return sorted_stats

    def display(self, sorted_stats, n=50):
        print(f"Tokens (Top {n}):")
        for count, token in sorted_stats[:n]:
            print(f"{token}: {count}")

doc = PreprocessedTextTokens("alice_wonderland.utf8.conll")

# Définir les catégories grammaticales à garder (par exemple, NNP pour les noms propres)
categories_to_keep = ["JJ"]

sorted_stats = doc.sorted_token_stat(categories_to_keep)
doc.display(sorted_stats)


Tokens (Top 50):
little: 111
other: 52
great: 39
large: 33
first: 33
last: 31
electronic: 27
next: 26
poor: 25
long: 25
same: 24
good: 24
old: 19
curious: 19
such: 18
right: 18
much: 18
sure: 16
full: 16
low: 14
many: 13
mad: 13
high: 13
whole: 12
small: 12
afraid: 12
glad: 11
own: 10
few: 10
different: 10
'Of: 10
white: 8
ready: 8
public: 8
free: 8
beautiful: 8
silent: 7
new: 7
moral: 7
golden: 7
deep: 7
bright: 7
'The: 7
sharp: 6
second: 6
nice: 6
melancholy: 6
http:/: 6
young: 5
wrong: 5


### Analyse de séquences
Pour avoir des informations plus intéressantes, on peut aussi regarder les séquences de 2 mots consécutifs.  
Ecrivez des fonctions pour compter toutes les séquences avec l'approche simple, et garder les plus "intéressantes"  
Généraliser pour compter des séquences de longueur arbitraire (fixée à l'avance). On appelle ces séquences de n mots des n-grammes (bigrammes pour n=2, trigrammes pour n=3, etc).  

Vous pouvez aller voir par curiosité l'inventaire historique fait par Google https://books.google.com/ngrams.

In [37]:
! pip install nltk

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 2.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 KB 6.0 MB/s eta 0:00:00a 0:00:01


In [38]:
import nltk
from nltk.util import ngrams
from nltk.probability import FreqDist

nltk.download("punkt")  # Télécharger les données nécessaires

class NgramAnalyzer:

    def __init__(self, filename):
        self.file = open(filename, 'r', encoding='utf-8').read()

    def count_ngrams(self, n):
        tokens = nltk.word_tokenize(self.file)
        n_grams = list(ngrams(tokens, n))
        fdist = FreqDist(n_grams)
        return fdist

    def sorted_ngram_stat(self, n, top_n=50):
        fdist = self.count_ngrams(n)
        return fdist.most_common(top_n)

    def display(self, sorted_stats):
        print(f"{n}-grams (Top {len(sorted_stats)}):")
        for ngram, count in sorted_stats:
            print(f"{ngram}: {count}")

doc = NgramAnalyzer("alice_wonderland_utf8.txt")

# Spécifiez la longueur de l'analyse des n-grammes (n)
n = 2  # Pour des bigrammes, par exemple

sorted_stats = doc.sorted_ngram_stat(n)
doc.display(sorted_stats)


[nltk_data] Downloading package punkt to /home/alexis/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2-grams (Top 50):
(',', 'and'): 450
(',', "'"): 429
("'", 'said'): 329
('!', "'"): 283
('.', "'"): 262
('said', 'the'): 206
("'", 'I'): 169
('?', "'"): 157
('of', 'the'): 127
('said', 'Alice'): 115
('in', 'a'): 95
(',', 'I'): 81
("'", 'the'): 81
('Alice', ','): 78
('in', 'the'): 76
('and', 'the'): 72
('to', 'the'): 69
('it', 'was'): 62
('the', 'Queen'): 62
(',', 'as'): 61
(',', 'but'): 60
('at', 'the'): 60
('it', ','): 57
('*', '*'): 57
('as', 'she'): 56
('a', 'little'): 56
("'", 'Alice'): 56
('she', 'had'): 55
('the', 'King'): 55
('Mock', 'Turtle'): 55
('I', "'m"): 54
('Alice', '.'): 54
('--', "'"): 52
(';', 'and'): 52
(',', 'you'): 51
('she', 'was'): 50
(',', 'she'): 50
('.', 'The'): 50
('to', 'be'): 50
("'", "'"): 50
('the', 'Gryphon'): 50
('the', 'Mock'): 49
('went', 'on'): 48
('.', 'Alice'): 47
("'", 'she'): 46
('do', "n't"): 46
('the', 'Hatter'): 46
('to', 'herself'): 45
(',', 'that'): 45
('and', 'she'): 43
